# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os
from pprint import pprint


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
# Reading csv file from directory
Cities_HW_df = pd.read_csv("../output_data/Cities_HW.csv")
Cities_HW_df

,name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,41.11,75,75,11.50,CL,1627161407
1,Ust'-Koksa,50.2696,85.6108,54.50,90,15,0.76,RU,1627161408
2,Port Elizabeth,-33.9180,25.5701,53.89,76,40,4.61,ZA,1627161408
3,Butaritari,3.0707,172.7902,83.71,73,100,15.32,KI,1627161409
4,Hilo,19.7297,-155.0900,81.63,76,90,6.91,US,1627161124
...,...,...,...,...,...,...,...,...,...
553,Ust-Nera,64.5667,143.2000,58.05,99,100,7.90,RU,1627161637
554,Cedral,23.8000,-100.7333,81.19,22,0,11.05,MX,1627161637
555,Gornopravdinsk,60.0500,69.9000,52.47,96,90,3.76,RU,1627161638
556,Elliot,-31.3333,27.8500,36.23,32,0,6.76,ZA,1627161638


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
# Configuring gmaps
gmaps.configure(api_key=g_key)

In [32]:
# Store latitude and longitude in locations
locations = Cities_HW_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
Humidity = Cities_HW_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(center = (46.0, -5.0), zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = Humidity, 
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
# Creating a new data frame to pull desired weather data

Ideal_City_df = pd.DataFrame(Cities_HW_df, columns = ['name', 'Lat', 'Lng', 'Country',
                                                      'Max Temp', 'Cloudiness',
                                                      'Humidity', 'Wind Speed'])

# Selecting range of Max Temp
temp_Ideal = (Ideal_City_df['Max Temp'] <= 78) & (Ideal_City_df['Max Temp'] >= 68)

# Selecting range of cloudiness
cloud_Ideal = Ideal_City_df['Cloudiness'] < 15

# Selecting range of humidity
humidity_Ideal = Ideal_City_df['Humidity'] < 55

# Selecting range of wind speed
wind_Ideal = Ideal_City_df['Wind Speed'] <= 15

# Concatenating columns
FitCity_df = Ideal_City_df[temp_Ideal & humidity_Ideal & cloud_Ideal & wind_Ideal]

FitCity_df

,name,Lat,Lng,Country,Max Temp,Cloudiness,Humidity,Wind Speed
8,Rikitea,-23.1203,-134.9692,PF,69.22,0,54,1.21
22,Fredericton,45.9454,-66.6656,CA,77.45,1,44,6.91
34,Unaí,-16.3575,-46.9061,BR,70.25,0,34,4.07
80,Bani Walid,31.7566,13.9942,LY,77.58,0,46,9.98
181,Granada,37.2500,-3.2500,ES,75.34,0,23,6.62
188,Kosonsoy,41.2561,71.5508,UZ,75.76,0,21,7.47
252,Corinth,37.9407,22.9573,GR,76.23,12,52,2.75
254,Aksay,47.2600,39.8700,RU,77.25,1,37,5.99
399,Henties Bay,-22.1160,14.2845,NaN,69.85,0,36,8.68
411,Gairo,39.8497,9.5047,IT,69.01,6,31,4.03


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
# Creating a new data frame (hotel_df) to store hotel data 

hotel_df = pd.DataFrame(FitCity_df, columns = ['name', 'Country',
                                                'Lat', 'Lng',
                                               'Max Temp', 'Cloudiness',
                                               'Humidity', 'Wind Speed'])

#hotel_df = FitCity_df
    
# Creating a new column, "Hotel Name" in hotel_df data frame
hotel_df['Hotel Name']= ''

# Renaming data frame column from 'name' to 'City'
hotel_df.rename(columns = {'name':'City'}, inplace = True)


hotel_df

,City,Country,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
8,Rikitea,PF,-23.1203,-134.9692,69.22,0,54,1.21,
22,Fredericton,CA,45.9454,-66.6656,77.45,1,44,6.91,
34,Unaí,BR,-16.3575,-46.9061,70.25,0,34,4.07,
80,Bani Walid,LY,31.7566,13.9942,77.58,0,46,9.98,
181,Granada,ES,37.2500,-3.2500,75.34,0,23,6.62,
188,Kosonsoy,UZ,41.2561,71.5508,75.76,0,21,7.47,
252,Corinth,GR,37.9407,22.9573,76.23,12,52,2.75,
254,Aksay,RU,47.2600,39.8700,77.25,1,37,5.99,
399,Henties Bay,NaN,-22.1160,14.2845,69.85,0,36,8.68,
411,Gairo,IT,39.8497,9.5047,69.01,6,31,4.03,


In [45]:
# Establishing base URL to call Google Maps API
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Setting parameters based on json object from Google Maps API documentation
params = {
    'radius': 5000,
    'type': 'hotel',
    'keyword': 'hotel',
    'key': g_key
}

# Setting up location variable as float
locations = hotel_df[["Lat", "Lng"]].astype(float)

# Using iterrows to iterate through pandas dataframe - it cycles through the data frame row by row
for index, row in hotel_df.iterrows():

    # Getting coordinates from df
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat}, {lng}'
    
    hotel_info = requests.get(base_url, params = params)
    
    hotel_info = hotel_info.json()
    
    # Selecting rows and setting up exceptions when pulling data from Google   
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_info['results'][0]['name']
        print(f'Hotel found')
    except:
        print('No hotel in this area.  Moving to next location')
        
print('-------------------------------------------------')
print('Data Retrieval Complete')
print('-------------------------------------------------')

Hotel found
Hotel found
Hotel found
Hotel found
No hotel in this area.  Moving to next location
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
-------------------------------------------------
Data Retrieval Complete
-------------------------------------------------


In [46]:
# Viewing data frame, noticed that there are blank spaces in the 'Hotel Name' column
hotel_df

,City,Country,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
8,Rikitea,PF,-23.1203,-134.9692,69.22,0,54,1.21,Pension Bianca & Benoit
22,Fredericton,CA,45.9454,-66.6656,77.45,1,44,6.91,"Radisson Kingswood Hotel & Suites, Fredericton..."
34,Unaí,BR,-16.3575,-46.9061,70.25,0,34,4.07,Halley Hotel
80,Bani Walid,LY,31.7566,13.9942,77.58,0,46,9.98,فندق الزيتونة
181,Granada,ES,37.2500,-3.2500,75.34,0,23,6.62,
188,Kosonsoy,UZ,41.2561,71.5508,75.76,0,21,7.47,Murodim
252,Corinth,GR,37.9407,22.9573,76.23,12,52,2.75,Hotel King Saron
254,Aksay,RU,47.2600,39.8700,77.25,1,37,5.99,"Park Otel' ""Kan'on"""
399,Henties Bay,NaN,-22.1160,14.2845,69.85,0,36,8.68,De Duine Hotel
411,Gairo,IT,39.8497,9.5047,69.01,6,31,4.03,Hotel Villa Selene


In [47]:
# Replacing the blank cells with 'NaN'
hotel_df['Hotel Name'].replace('', np.nan, inplace = True)

In [48]:
# Removing all rows that contain values with 'NaN'
hotel_df.dropna(subset = ['Hotel Name'], inplace = True)

In [49]:
# Displaying data frame to verify that the 'NaN' rows have been removed
hotel_df

,City,Country,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
8,Rikitea,PF,-23.1203,-134.9692,69.22,0,54,1.21,Pension Bianca & Benoit
22,Fredericton,CA,45.9454,-66.6656,77.45,1,44,6.91,"Radisson Kingswood Hotel & Suites, Fredericton..."
34,Unaí,BR,-16.3575,-46.9061,70.25,0,34,4.07,Halley Hotel
80,Bani Walid,LY,31.7566,13.9942,77.58,0,46,9.98,فندق الزيتونة
188,Kosonsoy,UZ,41.2561,71.5508,75.76,0,21,7.47,Murodim
252,Corinth,GR,37.9407,22.9573,76.23,12,52,2.75,Hotel King Saron
254,Aksay,RU,47.2600,39.8700,77.25,1,37,5.99,"Park Otel' ""Kan'on"""
399,Henties Bay,NaN,-22.1160,14.2845,69.85,0,36,8.68,De Duine Hotel
411,Gairo,IT,39.8497,9.5047,69.01,6,31,4.03,Hotel Villa Selene


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))